# Задание
Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный).

# Решение

## Парсинг

блок импорта

In [130]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date, timedelta
from pymongo import MongoClient
import unicodedata
import time

headless режим поддерживается только в chrome, но сохраню на всякий случай

In [ ]:
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')

инициализация драйвера. предварительно в консоли нужно ввести следующие команды:
```safaridriver --enable
safaridriver --port 0```

In [91]:
driver = webdriver.Safari()

открываем почту и логинимся

In [93]:
driver.get('https://mail.ru')

login = driver.find_element_by_id('mailbox:login')
login.click()
login.send_keys('study.ai_172@mail.ru')
login.send_keys(Keys.RETURN)
time.sleep(1)

paswd = driver.find_element_by_id('mailbox:password')
paswd.click()
paswd.send_keys('NewPassword172')
paswd.send_keys(Keys.RETURN)

находим ссылку на первое письмо

In [125]:
driver.get('https://m.mail.ru/inbox/')
WebDriverWait(driver,10).until(EC.presence_of_element_located((
   By.CLASS_NAME, 'msglist'
)))
letter_link = driver.find_element(By.CSS_SELECTOR, 'table.msglist a.messageline__link').get_attribute('href')
letter_link

'https://m.mail.ru/message/15796784172116936300'

функция парсинга письма

In [134]:
def scrap_letter(start_page: str):
    driver.get(start_page)
    WebDriverWait(driver,10).until(EC.presence_of_element_located((
        By.CLASS_NAME, 'footer'
    )))
    data = {
        'url': driver.current_url,
        'header': driver.find_element(By.CSS_SELECTOR, 'span.readmsg__theme').text.replace('\n', '').replace('\t', ''),
        'sender': driver.find_element(By.CSS_SELECTOR, 'span.readmsg__addressed-word ~ a').text,
        'date': driver.find_element(By.CSS_SELECTOR, 'span.readmsg__mail-date').text,
        'text': unicodedata.normalize('NFKD', driver.find_element(By.ID, 'readmsg__body').text).replace('\n', ' ').replace('\t', ' ').replace('  ', ''),
    }
    next_link = driver.find_elements(By.CSS_SELECTOR, 'div.readmsg__horizontal-block__right-block > a.readmsg__text-link')
    if (next_link):
        return data, next_link[0].get_attribute('href')
    else:
        return data, None


scrap_letter(letter_link)

({'url': 'https://m.mail.ru/message/15796784172116936300',
  'header': 'Обзор IT-новостей 22 января',
  'sender': 'Tproger Live <news@tproger.ru>',
  'date': 'Срд, 22 Янв 2020 10:33',
  'text': "Отказ от шифрования в iCloud и бесплатный доступ к «Госуслугам»Шифрование iCloudReuters узнал, что Apple какое-то время назад отказалась от идеи оконечного шифрования бэкапов в iCloud. Специалистам из проекта Plesio (или KeyDrop), которые занимались этой функцией, сказали разработку прекратить. Источники в ФБР и самой компании намекнули, что тому способствовало давление со стороны властей. Если бы проект с шифрованием реализовали, Apple потеряла бы доступ к данным, а значит, его лишились бы и следователи.На днях мы рассказывали об истории, в которой на компанию давили из-за разблокировки нескольких iPhone. Её обвиняли в бездействии, и Apple ответила, что резервные копии смартфонов она ФБР всё-таки передала. Наверное, следователям обидно было бы терять такой богатый источник информации.Впрочем, 

итератор по всем письмам

In [127]:
class Scrapper:
    def __init__(self, start_page: str):
        self.next = start_page
        self.data = None
        self.total = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.next:
            self.total += 1
            self.data, self.next = scrap_letter(self.next)
            return self.data
        else:
            raise StopIteration

## Сохраняем результаты

стартуем образ с mongodb

In [128]:
!docker start scrapy_mongo

scrapy_mongo


открываем первое письмо и начинаем парсить, пока не дойдём до последнего, сохраняя результат в mongodb. до первого совпадения

In [131]:
client = MongoClient('localhost', 27017)
db = client.selenium
collection = db['email']

In [138]:
scrapper = Scrapper(letter_link)
for letter in scrapper:
    print(letter['header'])
    if collection.update_one({'_id': letter['url']}, {'$set': letter}, upsert=True).matched_count != 0:
        break
print(scrapper.total)

Обзор IT-новостей 22 января
 Вход с нового устройства в аккаунт
Личное приглашение
Обзор IT-новостей 21 января
Обзор IT-новостей 20 января
Обзор IT-новостей 17 января
🔥 Игры для программистов
Обзор IT-новостей 16 января
Обзор IT-новостей 15 января
Подарок на наш День рождения
Обзор IT-новостей 14 января
Обзор IT-новостей 13 января
Программистское чтиво
Обзор IT-новостей 10 января
Обзор IT-новостей 9 января
Обзор IT-новостей 8 января
Обзор IT-новостей 7 января
Просто полезное письмо
Последнее письмо
Обзор IT-новостей 31 декабря
Обзор IT-новостей 30 декабря
Последний шанс
У вас 159 писем за год
Последний iPhone ждет вас
Самый последний шанс
Обзор IT-новостей 27 декабря
ТОП6 курсов для вас
Обзор IT-новостей 26 декабря
Обзор IT-новостей 25 декабря
Интересная статистика по зарплатам
Обзор IT-новостей 24 декабря
Обзор IT-новостей 23 декабря
Обзор IT-новостей 20 декабря
Есть бесплатное обучение. Сможете найти?
GeekBrains
Обзор IT-новостей 19 декабря
Обзор IT-новостей 18 декабря
Вечерний обзор

## Завершаем работу

останавливаем mongodb

In [139]:
!docker stop scrapy_mongo

scrapy_mongo


завершаем работу драйвера selenium

In [140]:
driver.quit()